In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
words = open('names.txt').read().splitlines()

chars = sorted(list(set(''.join(words))))
stoi = {c: i + 1 for i, c in enumerate(chars)}
stoi['.'] = 0
print(stoi)
itos = {i: c for c, i in stoi.items()}
# itos = {i: ch for ch, i in stoi.items()}
print(itos)

def buid_dataset(words):
    # build dataset
    block_size = 3
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            # print(''.join(itos[i] for i in context), '----->', itos[ix])
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words)) 
n2 = int(0.9 * len(words))

Xtr, Ytr = buid_dataset(words[:n1])
Xdev, Ydev = buid_dataset(words[n1:n2])
Xte, Yte = buid_dataset(words[n2:])
Xtr.shape, Xdev.shape, Xte.shape

{'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, '.': 0}
{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


(torch.Size([182625, 3]), torch.Size([22655, 3]), torch.Size([22866, 3]))

In [3]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | maxdiff: {maxdiff:.1e}')

In [5]:
n_embd = 10 # embedding size
n_hidden = 200 # hidden layer size
block_size = 3

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((27, n_embd), generator=g)
W1 = torch.randn((block_size * n_embd, n_hidden), generator=g) * (5/3) / (n_embd * block_size) ** 0.5
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, 27), generator=g) * 0.1
b2 = torch.randn(27, generator=g) * 0.0

bngain = torch.ones((1, n_hidden)) * 0.1 + 1
bnbias = torch.zeros((1, n_hidden)) * 0.1
bnmean_running = torch.zeros((1, n_hidden))
bnstd_running = torch.ones((1, n_hidden))

parameters = [C, W1, W2, b2, bngain, bnbias]
sum(p.nelement() for p in parameters)
for p in parameters:
    p.requires_grad = True

In [6]:
batch_size = 32
n = batch_size
# construct a minbatch
ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [13]:
# forward pass
emb = C[Xb] # embed the character into vectors
embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
hprebn = embcat @ W1 + b1 # pre-activation
# batch normalization
bnmeani = hprebn.sum(0, keepdim=True) * (n ** -1)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = bndiff2.sum(0, keepdim=True) * (1 /(n - 1)) # unbiased variance
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bnraw * bngain + bnbias
# activation
h = torch.tanh(hpreact)
logits = h @ W2
# cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1
probs = counts * counts_sum_inv
logprobs = probs
loss = -logprobs[range(n), Yb].mean()
